<a href="https://colab.research.google.com/github/Tealbox-Digital/KodeKloud_rolling_window_PoCs/blob/main/Purchase_Analysis_across_rolling_window.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Run This To Install The Libraries and Connect To The Big Query and Allow All The Permissions Asked In The Dialog Box That Operns After Running The Cell

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.cloud import bigquery
client = bigquery.Client(project='kk-data-analytics')
import pandas as pd

#1. A simple Step-By_Step Process to get the Purchases for Multiple Rolling Window range

a. Enter the Date Range you want to get the Data for

----------------------------------Caution--------------------------------

the end date is not included in the analysis, for example if you want the data from 1 to 7 than put the start date as 1 and the end date as 8, here 8 wont be included in the analysis, so keep this is mind while entering the date range

In [ ]:
start_date = input("Enter start date (YYYY-MM-DD): ")
end_date = input("Enter end date (YYYY-MM-DD): ")

Enter start date (YYYY-MM-DD): 2025-06-24
Enter end date (YYYY-MM-DD): 2025-07-01


b. Run the below two cells to get an idea about what are the columns in the leads table and how many entries are present in the time range you selected

ASSUMPTIONS TAKEN::

We are excludiung the values where the email of the leads in not present

In [ ]:
query = f"""
WITH enriched_leads AS (
  SELECT *,
    CASE
      WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_') THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_') THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_') THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_first,
    CASE
      WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_') THEN 'India'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_') THEN 'US'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_') THEN 'UK'
      WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
      ELSE 'TBU'
    END AS geography_last
  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
  WHERE lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
  AND useremail IS NOT NULL
  AND useremail != ''
)

SELECT DISTINCT
  useremail,
  lead_submission_date,
  lead_utm_campaign_first,
  lead_utm_campaign_latest,
  utm_source_first,
  utm_source_latest,
  is_new_addition,
  CASE
    WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
    WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
    WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
    ELSE geography_first
  END AS Geography
FROM enriched_leads
"""
df_filters = client.query(query).to_dataframe()
df_filters.head(20)

,useremail,lead_submission_date,lead_utm_campaign_first,lead_utm_campaign_latest,utm_source_first,utm_source_latest,is_new_addition,Geography
0,atnere2022@gmail.com,2025-07-21 03:50:44+00:00,None,None,None,None,1,TBU
1,makhlouf.ayoub@etu.uae.ac.ma,2025-07-15 03:32:26+00:00,None,None,None,None,1,TBU
2,haythamabdelaty@gmail.com,2025-07-15 19:48:03+00:00,None,None,None,None,1,TBU
3,kkotak.neo@gmail.com,2025-07-19 22:33:01+00:00,None,None,None,None,1,TBU
4,bharathvamaravelli@gmail.com,2025-07-21 04:29:20+00:00,None,None,None,None,1,TBU
5,rglanish@gmail.com,2025-07-18 12:14:13+00:00,None,None,None,None,1,TBU
6,pavanp.2546@gmail.com,2025-07-21 01:59:58+00:00,None,None,None,None,1,TBU
7,mo.foad44@gmail.com,2025-07-20 09:57:29+00:00,None,None,None,None,1,TBU
8,ralfgraefe1@gmail.com,2025-07-18 13:08:28+00:00,None,None,None,None,1,TBU
9,ritikaraisinghani09@gmail.com,2025-07-18 19:01:01+00:00,None,None,chatgpt.com,chatgpt.com,1,TBU


In [ ]:
df_filters.shape      #this tells us (no. of rows, no. of columns)

(4940, 8)

* GETTING ALL THE AVAILABLE UNIQUE CAMPAIGNS AND OTHER VALUES AVAILABLE IN THE DATA, SO THAT USER CAN SELECT WHAT ALL THE FILTERS HE/SHE WANTS TO APPLY ON THE DATA

* SELECT FROM THE LIST BELOW, ALSO TO SELECT MULTIPLE OPTIONS, CLICK ON THE OPTIONS WHILE PRESSING THE "ctrl" KEY

* ALSO HERE IF YOU SEE 'NONE' OR '(DIRECT)' IN THE OPTIONS FOR THE lead_utm_campaign_first THEN IT MEANS THEY ARE ORGANIC LEADS

* IF YOU SEE BOTH OF THEM SELECT BOTH OF THEM FOR THE ANALYSIS RELATED TO THE ORGANIC LEADS

* IN THE 'is_new_addition' COLUMN '1' REPRESENTS THAT IF YOU WANT TO NEW LEADS to KK Universe AND '0' FOR THE LEADS THAT ARE NOT NEW (they have filled leads historically)


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# Dictionary to store widgets for each column
selectors = {}

# Create and display widgets
for col in df_filters.columns:
    # Get unique values including None
    unique_vals = df_filters[col].unique().tolist()

    # Sort non-None values and add None at the beginning if it exists
    if None in unique_vals:
        unique_vals.remove(None)
        unique_vals = sorted(unique_vals)
        unique_vals.insert(0, None) # Add None at the beginning
    else:
        unique_vals = sorted(unique_vals)

    multi_select = widgets.SelectMultiple(
        options=unique_vals,
        description=col,
        rows=min(10, len(unique_vals)),
        layout=widgets.Layout(width='60%')
    )
    selectors[col] = multi_select
    display(multi_select)

SelectMultiple(description='useremail', layout=Layout(width='60%'), options=('0708vineeth@gmail.com', '0898jon…

SelectMultiple(description='lead_submission_date', layout=Layout(width='60%'), options=(Timestamp('2025-07-15 …

SelectMultiple(description='lead_utm_campaign_first', layout=Layout(width='60%'), options=(None, '11549847-New…

SelectMultiple(description='lead_utm_campaign_latest', layout=Layout(width='60%'), options=(None, '15030928-fr…

SelectMultiple(description='utm_source_first', layout=Layout(width='60%'), options=(None, '(direct)', 'Home', …

SelectMultiple(description='utm_source_latest', layout=Layout(width='60%'), options=(None, 'Home', 'HubSpot', …

SelectMultiple(description='is_new_addition', layout=Layout(width='60%'), options=(0, 1), rows=2, value=())

SelectMultiple(description='Geography', layout=Layout(width='60%'), options=('India', 'ROTW', 'TBU', 'UK', 'US…

Run the below table to know what filters you have selected

In [ ]:
## Create a dictionary to hold selected values
selected_filters = {}

# Loop through each widget and get the selected values
for col, widget in selectors.items():
    selected_filters[col] = list(widget.value)  # Convert from tuple to list

# Example: print them all
for col, selected_values in selected_filters.items():
    print(f"{col}: {selected_values}")
#print(selected_filters.lead_utm_campaign_first)

useremail: []
lead_submission_date: []
lead_utm_campaign_first: ['KK_GDN_IN_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA', 'KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_PMAX_IN_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Free-Week-2025_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Kubernetes_Email_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Sale-Campaign_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Sale-Campaign_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_PMAX_US_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_SEM_IN_B2C-Brand_MIX_B_PROS_Air Cover', "KK_SEM_IN_B2C-Brand_PM_B_PROS_NOA_SEP'24", 'KK_SEM_IN_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', "KK_SEM_IN_B2C-Generic-DevOps_PM_B_PROS_NOA_SEP'24", "KK_SEM_IN_B2C-Generic-Software&Tools

Here we are converting the filters into the sql query

Running the below cell will tell you how many total leads are there and how many of then are unique

In [ ]:
# 1) Build your dynamic filter string exactly as before:
selected_filters = { col: list(widget.value) for col, widget in selectors.items() }

where_clauses = []
for col, values in selected_filters.items():
    if not values:
        continue

    sql_literals = []
    include_null_check = False

    for v in values:
        if v is None:
            include_null_check = True
        elif isinstance(v, str):
            safe = v.replace("'", "\\'")
            sql_literals.append(f"'{safe}'")
        else:
            sql_literals.append(str(v))

    column_clauses = []
    if sql_literals:
        column_clauses.append(f"{col} IN ({', '.join(sql_literals)})")

    if include_null_check:
        column_clauses.append(f"{col} IS NULL")

    if column_clauses:
        where_clauses.append(f"({' OR '.join(column_clauses)})")


filter_sql = " AND ".join(where_clauses) if where_clauses else "1=1"
print(filter_sql)
# 2) Plug into this two‐step CTE query:
query = f"""
WITH leads_base AS (
      SELECT
        id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest ,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
        DATE(lead_submission_date) AS lead_date,
        -- geography_first
        CASE
          WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_first,
        -- geography_last
        CASE
          WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_last
      FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
      WHERE
        lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
        AND useremail IS NOT NULL AND useremail != ''
    ),

    enriched_leads AS (
      SELECT
         id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest ,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
       lead_date,
        CASE
          WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
          WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
          WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
          ELSE geography_first
        END AS Geography
      FROM leads_base
    ),

    filtered_leads AS (
      SELECT distinct
        id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest ,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
        lead_date,
        Geography
      FROM enriched_leads
      WHERE {filter_sql}
    ),

    cohort_stats AS (
      SELECT COUNT(*) AS total_leads
      FROM filtered_leads
    )


SELECT
  COUNT(*)               AS total_leads,
  COUNT(DISTINCT useremail) AS unique_leads

FROM filtered_leads


"""
# print(query)
# 3) Execute
df_leads = client.query(query).to_dataframe()

(lead_utm_campaign_first IS NULL)


In [ ]:
df_leads.head()

,total_leads,unique_leads
0,4,4


Now we will get the purchase data in the rolling window, just set up the rolling window values in daya according to what you want

You can select multiple windows at the same time just put them in the comma separated way like below:

windows = [7, 14, 28, 30, 60, 90, 120]

ASSUMPTIONS TAKEN::

we are removing the values where the email is not present of a user and also if the purchase subtotal is negative, we wont be including them in the analysis

In [ ]:
def compute_purchase_counts(window_days,filter_sql):
    query = f"""
    WITH leads_base AS (
      SELECT
        id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest ,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
        DATE(lead_submission_date) AS lead_date,
        -- geography_first
        CASE
          WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_first,
        -- geography_last
        CASE
          WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_last
      FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
      WHERE
        lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
        AND useremail IS NOT NULL AND useremail != ''
    ),

    enriched_leads AS (
      SELECT
         id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest ,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
       lead_date,
        CASE
          WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
          WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
          WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
          ELSE geography_first
        END AS Geography
      FROM leads_base
    ),

    filtered_leads AS (
      SELECT distinct
        id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest ,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
        lead_date,
        Geography
      FROM enriched_leads
      WHERE {filter_sql}
    ),

    cohort_stats AS (
      SELECT COUNT(*) AS total_leads
      FROM filtered_leads
    ),

    purchases_window AS (
      SELECT distinct
        fl.id,fl.useremail, r.received_at,r.subtotal_USD, fl.lead_utm_campaign_first,fl.is_new_addition,
        fl.utm_source_first,
        fl.utm_source_latest
        -- COUNT(r.customer_id) AS purchases_count
        -- COALESCE(SUM(r.subtotal_USD),0) AS purchases_revenue
      FROM filtered_leads fl
     JOIN `kk-data-analytics.TB_Customer_Analytics.Revenue_2025_v1_4` r
        ON r.customer_id = fl.id
       AND DATE(r.received_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)
       AND r.subtotal_USD > 0

    ),

    sl as (SELECT
      COUNT(*)               AS purchases_count,        -- one per DISTINCT row above
      SUM(subtotal_USD)      AS purchases_revenue     -- sum over those distinct rows
      FROM purchases_window pw
      )


select
 {window_days} AS window_days,
 * ,
 SAFE_DIVIDE(sl.purchases_count, cs.total_leads) * 100          AS conversion_rate,
      SAFE_DIVIDE(sl.purchases_revenue, sl.purchases_count)          AS avg_revenue_per_purchase
 from sl cross join cohort_stats as cs
    """
    return client.query(query).to_dataframe()

# ─── EXECUTE FOR MULTIPLE WINDOWS ──────────────────────────────────────────────
windows = [7, 14, 28, 30, 60, 90, 120]
results = []
print({filter_sql})
for w in windows:
    df = compute_purchase_counts(w,filter_sql)
    results.append(df)

final_df = pd.concat(results, ignore_index=True)
display(final_df)

# ─── OPTIONAL: EXPORT TO CSV ───────────────────────────────────────────────────
final_df.to_csv("rolling_window_purchase_counts.csv", index=False)
print("✅ CSV exported to rolling_window_purchase_counts.csv")

{"lead_utm_campaign_first IN ('KK_B2C_IN_DG_DevOpsTools', 'KK_GDN_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA_APR\\'25-TB', 'KK_GDN_IN_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_GDN_IN_B2C-Free-Week-2025_CONV_NAN_RTG_MULT_NA', 'KK_GDN_IN_B2C-Generic-DevOps_PM_NB_PROS_NOA_APR\\'25-TB', 'KK_GDN_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', 'KK_GDN_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA_APR\\'25-TB', 'KK_GDN_IN_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA', 'KK_GDN_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA_30_APR\\'25-TB', 'KK_GDN_ROTW_B2C-Free-Week-2025_CONV_NAN_RTG_MULT_NA', 'KK_GDN_ROTW_B2C-Generic-DevOps_PM_NB_PROS_NOA_30_APR\\'25-TB', 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_GDN_ROTW_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA_30_APR\\'25-TB', 'KK_GDN_ROTW_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA', 'KK_GDN_US/IN/ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA_APR\\'25-TB', 'KK_GDN_US/IN/ROTW_B2C-Generic-DevOps_PM_NB_PROS_NOA_APR\\'25-TB', 'KK_GDN_US/IN/ROTW_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA_APR\\

,window_days,purchases_count,purchases_revenue,total_leads,conversion_rate,avg_revenue_per_purchase
0,7,791,83251.706021,26160,3.023700,105.248680
1,14,868,90412.618928,26160,3.318043,104.162003
2,28,939,97083.977114,26160,3.589450,103.390817
3,30,963,98334.337539,26160,3.681193,102.112500
4,60,1134,106650.438432,26160,4.334862,94.048006
5,90,1223,111367.377852,26160,4.675076,91.060816
6,120,1280,115484.009668,26160,4.892966,90.221883


✅ CSV exported to rolling_window_purchase_counts.csv


Now instead of getting the combined purchase data for all the 'lead_utm_campaign_first' selected earlier, if you want to get their individual purchases in the same date range run the below command

if you want to run it for any other filter just change the column name here:
"for item in selected_filters['lead_utm_campaign_first']:"

and select those columns from the drop down filters above and run the cell below



In [ ]:
def compute_purchase_counts(window_days, filter_value):
    # Ensure filter_value is a string with single quotes around it and handle escaping
    if isinstance(filter_value, str):
        safe = filter_value.replace("'", r"\'")
        # now safe looks like: KK_SEM_IN_B2C‑Brand_PM_B_PROS_NOA_SEP\'24
        filter_sql = f"('{safe}')" # Modified to format for the IN clause
    else:
        filter_sql = f"({str(filter_value)})" # Modified to format for the IN clause


    query = f"""

 WITH leads_base AS (
      SELECT
        id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest ,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
        DATE(lead_submission_date) AS lead_date,
        -- geography_first
        CASE
          WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_first,
        -- geography_last
        CASE
          WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_last
      FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
      WHERE
        lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
        AND useremail IS NOT NULL AND useremail != ''
    ),

    enriched_leads AS (
      SELECT DISTINCT
        *,
        CASE
          WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
          WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
          WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
          ELSE geography_first
        END AS Geography
      FROM leads_base
    ),

    filtered_leads AS (
      SELECT *
      FROM enriched_leads
     WHERE lead_utm_campaign_first IN {filter_sql}
    ),

    cohort_stats AS (
      SELECT COUNT(*) AS total_leads
      FROM filtered_leads
    ),

    purchases_window AS (
      SELECT distinct
        fl.id,fl.useremail, r.received_at,r.subtotal_USD
        -- COUNT(r.customer_id) AS purchases_count
        -- COALESCE(SUM(r.subtotal_USD),0) AS purchases_revenue
      FROM filtered_leads fl
     JOIN `kk-data-analytics.TB_Customer_Analytics.Revenue_2025_v1_4` r
        ON r.customer_id = fl.id
       AND DATE(r.received_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)
       AND r.subtotal_USD > 0

    ),

    sl as (SELECT
      COUNT(*)               AS purchases_count,        -- one per DISTINCT row above
      SUM(subtotal_USD)      AS purchases_revenue     -- sum over those distinct rows
      FROM purchases_window pw
      )


select
 {window_days} AS window_days,
 * ,
 SAFE_DIVIDE(sl.purchases_count, cs.total_leads) * 100          AS conversion_rate,
      SAFE_DIVIDE(sl.purchases_revenue, sl.purchases_count)          AS avg_revenue_per_purchase
 from sl cross join cohort_stats as cs
    """
    return client.query(query).to_dataframe()



print(f"Between dates {start_date} and {end_date}, excluding {end_date}")
i=1
sum=0
print(len(selected_filters['lead_utm_campaign_first']))
for item in selected_filters['lead_utm_campaign_first']:
  windows = [28]
  results = []
  for w in windows:
    df = compute_purchase_counts(w, item)
    results.append(df)

  final_df = pd.concat(results, ignore_index=True)
  print(f"{i}----- {item}\n")
  sum=final_df['total_leads'][0]+sum
  print(f"sum is=",sum)
  i=i+1
  display(final_df)


Between dates 2025-06-24 and 2025-07-01, excluding 2025-07-01
26
1----- KK_GDN_IN_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA

sum is= 24


,window_days,purchases_count,purchases_revenue,total_leads,conversion_rate,avg_revenue_per_purchase
0,28,0,NaN,24,0.0,NaN


2----- KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB

sum is= 46


,window_days,purchases_count,purchases_revenue,total_leads,conversion_rate,avg_revenue_per_purchase
0,28,0,NaN,22,0.0,NaN


3----- KK_PMAX_IN_B2C-DevOps_CONV_B_PROS_MULT_NA

sum is= 46


...

,window_days,purchases_count,purchases_revenue,total_leads,conversion_rate,avg_revenue_per_purchase
0,28,0,NaN,7,0.0,NaN


25----- KK_SEM_US_B2C-Kubernetes-AB_MIX_NB_PROS_DG

sum is= 819


,window_days,purchases_count,purchases_revenue,total_leads,conversion_rate,avg_revenue_per_purchase
0,28,1,250.0,1,100.0,250.0


26----- KK_YT_IN_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA

sum is= 821


,window_days,purchases_count,purchases_revenue,total_leads,conversion_rate,avg_revenue_per_purchase
0,28,1,140.796,2,50.0,140.796


#2. A quick way to do when you want a specific campaigns, this will give you the purchases in the rolling window directly

enter the date, remeber that the end date would not be included in the analysis so you may have to put one day more than the date range you want to analyze

In [ ]:
start_date = input("Enter start date (YYYY-MM-DD): ")
end_date = input("Enter end date (YYYY-MM-DD): ")

Enter start date (YYYY-MM-DD): 2025-01-01
Enter end date (YYYY-MM-DD): 2025-07-18


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Layout

# run your distinct‐campaigns query (assumes start_date/end_date & 'client' already defined)
#put the keywords with LIKE OR NOT LIKE in the query below to get the options for the campaigns containing those keywords
query = f"""
  SELECT DISTINCT lead_utm_campaign_first
  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
  WHERE lead_utm_campaign_first LIKE 'KK%'  AND lead_utm_campaign_first  LIKE '%Cloud%'
  --AND lead_utm_campaign_first NOT LIKE '%Sale%'
    AND lead_submission_date
        BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}');
"""
df_new_filters = client.query(query).to_dataframe()

# prepare the selectors dict
selectors = {}

# 1) Campaigns multi‐select
campaign_options = df_new_filters['lead_utm_campaign_first'].tolist()
campaign_widget = widgets.SelectMultiple(
    options=campaign_options,
    value=campaign_options,    # pre‑select all campaigns by default
    description='Campaigns',
    rows=min(10, len(campaign_options)),
    layout=Layout(width='60%')
)
display(campaign_widget)
selectors['lead_utm_campaign_first'] = campaign_widget

# 2) is_new_addition multi‐select
#    (you can pick 0, 1, both or neither)
new_addition_widget = widgets.SelectMultiple(
    options=[0, 1],
    value=[],                  # none selected by default → no filter on this field
    description='New Lead?',
    rows=2,
    layout=Layout(width='30%')
)
display(new_addition_widget)
selectors['is_new_addition'] = new_addition_widget


SelectMultiple(description='Campaigns', index=(0,), layout=Layout(width='60%'), options=(None,), rows=1, value…

SelectMultiple(description='New\xa0Lead?', layout=Layout(width='30%'), options=(0, 1), rows=2, value=())

run this to convert the selected options into the query

In [ ]:
selected_filters = { col: list(widget.value) for col, widget in selectors.items() }

where_clauses = []
for col, values in selected_filters.items():
    if not values:
        continue
    literals = []
    for v in values:
        if isinstance(v, str):
            safe = v.replace("'", "\\'")
            literals.append(f"'{safe}'")
        else:
            literals.append(str(v))
    where_clauses.append(f"{col} IN ({', '.join(literals)})")

filter_sql = " AND ".join(where_clauses) if where_clauses else "1=1"
print(filter_sql)

lead_utm_campaign_first IN ('KK_SEM_IN_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_PMAX_US_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_SEM_ROTW_B2C-Cloud-AB_MIX_NB_PROS_DG-TB') AND is_new_addition IN (1)


 enter the window range you want the output for.

 FOR EXAMPLE: windows = [7,14,21,28]

 This will give you combined purchase date for all the campaigns together

In [ ]:
def compute_purchase_counts(window_days,filter_sql):
    query = f"""
    WITH leads_base AS (
      SELECT
        id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest ,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
        DATE(lead_submission_date) AS lead_date,
        -- geography_first
        CASE
          WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_first,
        -- geography_last
        CASE
          WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_last
      FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
      WHERE
        lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
        AND useremail IS NOT NULL AND useremail != ''
    ),

    enriched_leads AS (
      SELECT
         id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest ,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
       lead_date,
        CASE
          WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
          WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
          WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
          ELSE geography_first
        END AS Geography
      FROM leads_base
    ),

    filtered_leads AS (
      SELECT distinct
        id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest ,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
        lead_date,
        Geography
      FROM enriched_leads
      WHERE {filter_sql}
    ),

    cohort_stats AS (
      SELECT COUNT(*) AS total_leads
      FROM filtered_leads
    ),

    purchases_window AS (
      SELECT distinct
        fl.id,fl.useremail, r.received_at,r.subtotal_USD, fl.lead_utm_campaign_first,fl.is_new_addition,
        fl.utm_source_first,
        fl.utm_source_latest
        -- COUNT(r.customer_id) AS purchases_count
        -- COALESCE(SUM(r.subtotal_USD),0) AS purchases_revenue
      FROM filtered_leads fl
     JOIN `kk-data-analytics.TB_Customer_Analytics.Revenue_2025_v1_4` r
        ON r.customer_id = fl.id
       AND DATE(r.received_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)
       AND r.subtotal_USD > 0

    ),

    sl as (SELECT
      COUNT(*)               AS purchases_count,        -- one per DISTINCT row above
      SUM(subtotal_USD)      AS purchases_revenue     -- sum over those distinct rows
      FROM purchases_window pw
      )


select
 {window_days} AS window_days,
 * ,
 SAFE_DIVIDE(sl.purchases_count, cs.total_leads) * 100          AS conversion_rate,
      SAFE_DIVIDE(sl.purchases_revenue, sl.purchases_count)          AS avg_revenue_per_purchase
 from sl cross join cohort_stats as cs
    """
    return client.query(query).to_dataframe()

# ─── EXECUTE FOR MULTIPLE WINDOWS ──────────────────────────────────────────────
windows = [7,14,21,28]# enter the window range you want the output for
results = []
print({filter_sql})
for w in windows:
    df = compute_purchase_counts(w,filter_sql)
    results.append(df)

final_df = pd.concat(results, ignore_index=True)
display(final_df)

# ─── OPTIONAL: EXPORT TO CSV ───────────────────────────────────────────────────
# final_df.to_csv("rolling_window_purchase_counts.csv", index=False)
# print("✅ CSV exported to rolling_window_purchase_counts.csv")

{'(lead_utm_campaign_first IS NULL)'}


,purchases_count,purchases_revenue,total_leads,conversion_rate,avg_revenue_per_purchase
0,9979,1.211107e+06,104849,9.517497,121.365606


#3. Multiple date ranges with their specific rolling windows

Select the campaigns by applying the sql filters correctly

In [ ]:
# =========================================
# 0. Imports & BigQuery client
# =========================================
import pandas as pd
from google.cloud import bigquery
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

client = bigquery.Client(project="kk-data-analytics")  # change project if needed


# =========================================
# 1. Campaign selector UI  (run this cell first)
# =========================================
campaign_query = """
  SELECT DISTINCT lead_utm_campaign_first
  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
  WHERE lead_utm_campaign_first LIKE 'KK%'  AND lead_utm_campaign_first LIKE '%Cloud%'
  -- AND lead_utm_campaign_first NOT LIKE '%Sale%'
  -- AND lead_submission_date BETWEEN TIMESTAMP('2025-01-01') AND TIMESTAMP('2025-07-23')
"""
df_new_filters = client.query(campaign_query).to_dataframe()

selectors = {}
campaign_options = df_new_filters['lead_utm_campaign_first'].dropna().tolist()

campaign_widget = widgets.SelectMultiple(
    options=campaign_options,
    value=campaign_options,               # preselect all
    description='Campaigns',
    rows=min(12, len(campaign_options)),
    layout=Layout(width='60%')
)
display(campaign_widget)
selectors['lead_utm_campaign_first'] = campaign_widget

new_addition_widget = widgets.SelectMultiple(
    options=[0, 1],
    value=[],                             # none selected => ignore this field
    description='New Lead?',
    rows=2,
    layout=Layout(width='20%')
)
display(new_addition_widget)
selectors['is_new_addition'] = new_addition_widget

def get_selected_filters():
    return {
        'lead_utm_campaign_first': list(selectors['lead_utm_campaign_first'].value),
        'is_new_addition': list(selectors['is_new_addition'].value)
    }


SelectMultiple(description='Campaigns', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17), …

SelectMultiple(description='New\xa0Lead?', layout=Layout(width='20%'), options=(0, 1), rows=2, value=())

See the options you have selected and verify them

In [ ]:
selected_filters = get_selected_filters()
print(selected_filters)

{'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-MarchTowardsCloud-Sale_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_SEM_IN_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_PMAX_US_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', 'KK_SEM_US_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', 'KK_SEM_ROTW_B2C-Cloud-AB_MIX_NB_PROS_DG-TB', "KK_GDN_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA_APR'25-TB", "KK_GDN_US/IN/ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA_APR'25-TB", 'KK_PMAX_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB', "KK_GDN_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA_30_APR'25-TB"], 'is_new_addition': [1]}


Specify the date ranges and rolling windows you want to analyze in the format of a Python list of dictionaries.

For example:

In [ ]:
date_windows_text = widgets.Textarea(
    value='[{"start": "2025-01-01", "end": "2025-07-07", "windows": [7, 14]}, {"start": "2025-02-01", "end": "2025-07-08", "windows": [7]}]',
    description='Date Ranges and Windows:',
    layout=Layout(width='80%', height='150px')
)

display(date_windows_text)

Textarea(value='[{"start": "2025-01-01", "end": "2025-07-07", "windows": [7, 14]}, {"start": "2025-02-01", "en…

Now run this cell to get the results for multiple date ranges and rolling windows

In [ ]:
# =========================================
# 3. Core query function (SQL body unchanged)
# =========================================
def compute_purchase_counts(window_days, filter_value, start_date, end_date):
    # Ensure filter_value is a string with single quotes around it and handle escaping
    if isinstance(filter_value, str):
        safe = filter_value.replace("'", r"\'")
        filter_sql = f"('{safe}')"  # IN (...) format
    else:
        filter_sql = f"({str(filter_value)})"

    query = f"""

 WITH leads_base AS (
      SELECT
        id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest ,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
        DATE(lead_submission_date) AS lead_date,
        -- geography_first
        CASE
          WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_first,
        -- geography_last
        CASE
          WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_last
      FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
      WHERE
        lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
        AND useremail IS NOT NULL AND useremail != ''
    ),

    enriched_leads AS (
      SELECT DISTINCT
        *,
        CASE
          WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
          WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
          WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
          ELSE geography_first
        END AS Geography
      FROM leads_base
    ),

    filtered_leads AS (
      SELECT *
      FROM enriched_leads
     WHERE lead_utm_campaign_first IN {filter_sql}
    ),

    cohort_stats AS (
      SELECT COUNT(*) AS total_leads
      FROM filtered_leads
    ),

    purchases_window AS (
      SELECT distinct
        fl.id,fl.useremail, r.received_at,r.subtotal_USD
      FROM filtered_leads fl
     JOIN `kk-data-analytics.TB_Customer_Analytics.Revenue_2025_v1_4` r
        ON r.customer_id = fl.id
       AND DATE(r.received_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)
       AND r.subtotal_USD > 0

    ),

    sl as (SELECT
      COUNT(*)               AS purchases_count,
      SUM(subtotal_USD)      AS purchases_revenue
      FROM purchases_window pw
      )

select
 {window_days} AS window_days,
 * ,
 SAFE_DIVIDE(sl.purchases_count, cs.total_leads) * 100          AS conversion_rate,
 SAFE_DIVIDE(sl.purchases_revenue, sl.purchases_count)          AS avg_revenue_per_purchase
 from sl cross join cohort_stats as cs
    """
    return client.query(query).to_dataframe()



# =========================================
# 4. Runner (no button – executed when this cell runs)
# =========================================
def run_batches():
    # parse date_windows from textarea safely (eval is okay in trusted env; otherwise use json.loads with valid JSON)
    try:
        date_windows = eval(date_windows_text.value, {}, {})
        assert isinstance(date_windows, list)
    except Exception as e:
        raise ValueError(f"Could not parse date_windows_text: {e}")

    selected_filters = get_selected_filters()

    i = 1
    grand_total = 0
    all_outputs = []   # collect ((start,end), campaign, df)

    for spec in date_windows:
        start_date = spec["start"]
        end_date   = spec["end"]
        windows    = spec["windows"]

        print(f"\n=== Between {start_date} and {end_date} (excluding {end_date}) ===")
        print(f"Campaign filters: {len(selected_filters['lead_utm_campaign_first'])}")

        for campaign in selected_filters['lead_utm_campaign_first']:
            dfs = [compute_purchase_counts(w, campaign, start_date, end_date) for w in windows]
            final_df = pd.concat(dfs, ignore_index=True)

            print(f"\n{i} ----- {campaign}")
            grand_total += final_df['total_leads'].iloc[0]
            print("Running total leads =", grand_total)
            display(final_df)

            all_outputs.append(((start_date, end_date), campaign, final_df.copy()))
            i += 1

    return all_outputs

# Run immediately
results = run_batches()
print("\nDone. `results` holds all DataFrames.")



=== Between 2025-01-01 and 2025-07-07 (excluding 2025-07-07) ===
Campaign filters: 10

1 ----- KK_PMAX_IN_B2C-MarchTowardsCloud-Sale_CONV_B_PROS_MULT_NA
Running total leads = 1


,window_days,purchases_count,purchases_revenue,total_leads,conversion_rate,avg_revenue_per_purchase
0,7,0,NaN,1,0.0,NaN
1,14,0,NaN,1,0.0,NaN



2 ----- KK_PMAX_IN_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB
Running total leads = 5440


,window_days,purchases_count,purchases_revenue,total_leads,conversion_rate,avg_revenue_per_purchase
0,7,16,1212.324419,5439,0.294172,75.770276
1,14,20,1519.107170,5439,0.367715,75.955359



3 ----- KK_SEM_IN_B2C-Cloud-AB_MIX_NB_PROS_DG-TB
Running total leads = 6797


...


19 ----- KK_PMAX_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA-TB
Running total leads = 17035


,window_days,purchases_count,purchases_revenue,total_leads,conversion_rate,avg_revenue_per_purchase
0,7,1,70.0,225,0.444444,70.0



20 ----- KK_GDN_ROTW_B2C-Cloud_CONV_NAN_PROS_MULT_NA_30_APR'25-TB
Running total leads = 17039


,window_days,purchases_count,purchases_revenue,total_leads,conversion_rate,avg_revenue_per_purchase
0,7,0,NaN,4,0.0,NaN



Done. `results` holds all DataFrames.


#4. For multiple dates ranges and combined of all campaigns and filters

In [ ]:
date_windows_text = widgets.Textarea(
    value='[{"start": "2025-07-16", "end": "2025-07-23", "windows": [7]}, {"start": "2025-07-09", "end": "2025-07-16", "windows": [14]}, {"start": "2025-07-02", "end": "2025-07-09", "windows": [21]}, {"start": "2025-06-25", "end": "2025-07-02", "windows": [28]}]',
    description='Date Ranges and Windows:',
    layout=Layout(width='80%', height='80px')
)

display(date_windows_text)

Textarea(value='[{"start": "2025-07-16", "end": "2025-07-23", "windows": [7]}, {"start": "2025-07-09", "end": …

In [ ]:
# =========================================
# 0. Imports & BigQuery client
# =========================================
import pandas as pd
from google.cloud import bigquery
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

client = bigquery.Client(project="kk-data-analytics")  # change project if needed


# =========================================
# 1. Campaign selector UI  (run this cell first)
# =========================================
campaign_query = """
  SELECT DISTINCT lead_utm_campaign_first
  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
  WHERE lead_utm_campaign_first LIKE 'KK%'
  AND lead_utm_campaign_first NOT LIKE '%FBA%' AND lead_utm_campaign_first LIKE '%PMAX%' AND lead_utm_campaign_first NOT LIKE '%Sale%'
  -- AND lead_submission_date BETWEEN TIMESTAMP('2025-01-01') AND TIMESTAMP('2025-07-23')
"""
df_new_filters = client.query(campaign_query).to_dataframe()

selectors = {}
campaign_options = df_new_filters['lead_utm_campaign_first'].dropna().tolist()

campaign_widget = widgets.SelectMultiple(
    options=campaign_options,
    value=campaign_options,               # preselect all
    description='Campaigns',
    rows=min(12, len(campaign_options)),
    layout=Layout(width='60%')
)
display(campaign_widget)
selectors['lead_utm_campaign_first'] = campaign_widget

new_addition_widget = widgets.SelectMultiple(
    options=[0, 1],
    value=[],                             # none selected => ignore this field
    description='New Lead?',
    rows=2,
    layout=Layout(width='20%')
)
display(new_addition_widget)
selectors['is_new_addition'] = new_addition_widget

def get_selected_filters():
    return {
        'lead_utm_campaign_first': list(selectors['lead_utm_campaign_first'].value),
        'is_new_addition': list(selectors['is_new_addition'].value)
    }


SelectMultiple(description='Campaigns', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1…

SelectMultiple(description='New\xa0Lead?', layout=Layout(width='20%'), options=(0, 1), rows=2, value=())

In [ ]:
selected_filters = get_selected_filters()
print(selected_filters)

{'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_PMAX_IN_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_UK_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_UK_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_UK_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_Generic-DevOps_CONV_B_PROS_MULT_NA', 

In [ ]:
# =========================================
# 3. Core query function (SQL body unchanged)
# =========================================
# =========================================
# 4. Core query function (SQL body minimally altered)
# =========================================
def compute_purchase_counts(window_days, start_date, end_date, extra_where):
    query = f"""
 WITH leads_base AS (
      SELECT
        id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
        DATE(lead_submission_date) AS lead_date,
        -- geography_first
        CASE
          WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_first,
        -- geography_last
        CASE
          WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_last
      FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
      WHERE
        lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
        AND useremail IS NOT NULL AND useremail != ''
    ),

    enriched_leads AS (
      SELECT DISTINCT
        *,
        CASE
          WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
          WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
          WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
          ELSE geography_first
        END AS Geography
      FROM leads_base
    ),

    filtered_leads AS (
      SELECT *
      FROM enriched_leads
      WHERE 1=1 {extra_where}
    ),

    cohort_stats AS (
      SELECT COUNT(*) AS total_leads
      FROM filtered_leads
    ),

    purchases_window AS (
      SELECT DISTINCT
        fl.id, fl.useremail, r.received_at, r.subtotal_USD
      FROM filtered_leads fl
      JOIN `kk-data-analytics.TB_Customer_Analytics.Revenue_2025_v1_4` r
        ON r.customer_id = fl.id
       AND DATE(r.received_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)
       AND r.subtotal_USD > 0
    ),

    sl AS (
      SELECT
        COUNT(*)          AS purchases_count,
        SUM(subtotal_USD) AS purchases_revenue
      FROM purchases_window pw
    )

SELECT
  {window_days} AS window_days,
  cs.total_leads,
  sl.purchases_count,
  SAFE_DIVIDE(sl.purchases_count, cs.total_leads) * 100 AS conversion_rate,
  sl.purchases_revenue,
  SAFE_DIVIDE(sl.purchases_revenue, sl.purchases_count) AS avg_revenue_per_purchase
FROM sl
CROSS JOIN cohort_stats cs
    """
    return client.query(query).to_dataframe()



# =========================================
# 3. Helper to build WHERE fragments
# =========================================
def _to_sql_in(values):
    """Return a BigQuery-safe IN (...) string for a Python list."""
    if not values:
        return None
    if isinstance(values[0], (int, float)):
        inner = ",".join(str(v) for v in values)
    else:
        # Use double quotes for string literals as requested
        inner = ",".join('"' + str(v).replace('"', '""') + '"' for v in values)
    return f"({inner})"

def build_extra_where(filters_dict):
    """
    Turn a dict like:
        {'lead_utm_campaign_first': [...], 'is_new_addition': [0,1]}
    into ANDed SQL fragments.
    """
    clauses = []
    for col, vals in filters_dict.items():
        if vals:
            in_clause = _to_sql_in(vals)
            clauses.append(f"{col} IN {in_clause}")
    if clauses:
        return " AND " + " AND ".join(clauses)
    return ""     # no extra filtering


# =========================================
# 4. Runner (no button – executed when this cell runs)
# =========================================
# =========================================
# 5. Runner – combined filters (no per-campaign loop)
# =========================================
def run_batches():
    # parse the date_windows text safely (eval is okay in trusted env; otherwise use json.loads with valid JSON)
    try:
        date_windows = eval(date_windows_text.value, {}, {})
        assert isinstance(date_windows, list)
    except Exception as e:
        raise ValueError(f"Could not parse date_windows_text: {e}")

    filters_dict = get_selected_filters()
    extra_where = build_extra_where(filters_dict)

    all_outputs = []
    for spec in date_windows:
        start_date = spec["start"]
        end_date   = spec["end"]
        windows    = spec["windows"]

        print(f"\n=== Between {start_date} and {end_date} ===")
        print("Filters applied:", filters_dict)

        dfs = [
            compute_purchase_counts(w, start_date, end_date, extra_where)
            for w in windows
        ]
        final_df = pd.concat(dfs, ignore_index=True)

        display(final_df)

        all_outputs.append(((start_date, end_date), final_df.copy()))

    return all_outputs

# Run immediately
results = run_batches()
print("\nDone. `results` holds each (date_range, df) tuple.")


=== Between 2025-07-16 and 2025-07-23 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_PMAX_IN_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_UK_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_UK_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_UK_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_NAN_PROS

,window_days,total_leads,purchases_count,conversion_rate,purchases_revenue,avg_revenue_per_purchase
0,7,110,2,1.818182,175.796,87.898



=== Between 2025-07-09 and 2025-07-16 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_PMAX_IN_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_UK_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_UK_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_UK_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_NAN_PROS

,window_days,total_leads,purchases_count,conversion_rate,purchases_revenue,avg_revenue_per_purchase
0,14,81,5,6.17284,631.991281,126.398256



=== Between 2025-07-02 and 2025-07-09 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_PMAX_IN_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_UK_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_UK_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_UK_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_NAN_PROS

,window_days,total_leads,purchases_count,conversion_rate,purchases_revenue,avg_revenue_per_purchase
0,21,335,3,0.895522,132.477764,44.159255



=== Between 2025-06-25 and 2025-07-02 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_PMAX_IN_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_UK_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_UK_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_UK_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_NAN_PROS

,window_days,total_leads,purchases_count,conversion_rate,purchases_revenue,avg_revenue_per_purchase
0,28,536,3,0.559701,310.912767,103.637589



Done. `results` holds each (date_range, df) tuple.


In [ ]:
# =========================================
# 0. Imports & BigQuery client
# =========================================
import pandas as pd
from google.cloud import bigquery
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

client = bigquery.Client(project="kk-data-analytics")  # change project if needed


# =========================================
# 1. Campaign selector UI  (run this cell first)
# =========================================
campaign_query = """
  SELECT DISTINCT lead_utm_campaign_first
  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
  WHERE lead_utm_campaign_first LIKE 'KK%'
  AND lead_utm_campaign_first NOT LIKE '%FBA%' AND lead_utm_campaign_first NOT LIKE '%PMAX%' AND lead_utm_campaign_first NOT LIKE '%Sale%'
  -- AND lead_submission_date BETWEEN TIMESTAMP('2025-01-01') AND TIMESTAMP('2025-07-23')
"""
df_new_filters = client.query(campaign_query).to_dataframe()

selectors = {}
campaign_options = df_new_filters['lead_utm_campaign_first'].dropna().tolist()

campaign_widget = widgets.SelectMultiple(
    options=campaign_options,
    value=campaign_options,               # preselect all
    description='Campaigns',
    rows=min(12, len(campaign_options)),
    layout=Layout(width='60%')
)
display(campaign_widget)
selectors['lead_utm_campaign_first'] = campaign_widget

new_addition_widget = widgets.SelectMultiple(
    options=[0, 1],
    value=[],                             # none selected => ignore this field
    description='New Lead?',
    rows=2,
    layout=Layout(width='20%')
)
display(new_addition_widget)
selectors['is_new_addition'] = new_addition_widget

def get_selected_filters():
    return {
        'lead_utm_campaign_first': list(selectors['lead_utm_campaign_first'].value),
        'is_new_addition': list(selectors['is_new_addition'].value)
    }


SelectMultiple(description='Campaigns', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1…

SelectMultiple(description='New\xa0Lead?', layout=Layout(width='20%'), options=(0, 1), rows=2, value=())

In [ ]:
selected_filters = get_selected_filters()
print(selected_filters)

{'lead_utm_campaign_first': ["KK_SEM_IN_B2C-Generic-DevOps_PM_B_PROS_NOA_SEP'24", 'KK_SEM_IN_B2C-Kubernetes-AB_MIX_NB_PROS_DG', "KK_SEM_IN_B2C-Brand_PM_B_PROS_NOA_SEP'24", "KK_SEM_IN_B2C-Generic-Software&Tools_PM_B_PROS_NOA_SEP'24", 'KK_SEM_NA_B2C-Kubernetes-AB_MIX_NB_PROS_DG', 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_GDN_IN_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_GDN_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', 'KK_YT_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', 'KK_SEM_IN_B2C-Non-Brand_MIX_NB_PROS_Air Cover', 'KK_SEM_IN_B2C-Early-Black-Friday_PM_NB_RTG_NOA', 'KK_SEM_IN_B2C-Goodbye-2024_PM_NB_RTG_NOA', 'KK_SEM_IN_B2C-Non-Brand-RLSA_MIX_NB_PROS_Air-Cover', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T2', 'KK_SEM_ROTW_B2C-Goodbye-2024_PM_NB_RTG_NOA_T2', "KK_SEM_UK_B2C-Brand_PM_B_PROS_NOA_SEP'24", 'KK_SEM_UK_B2C-Kubernetes-AB_MIX_NB_PROS_DG', "KK_SEM_US_B2C-Generic-DevOps_PM_NB_PROS_NOA_SEP'24", 'KK_SEM_US_B2C-Goodbye-2024_PM_NB_RTG_NOA', 'KK_YT_IN_B2C-Earl

In [ ]:
# =========================================
# 3. Core query function (SQL body unchanged)
# =========================================
# =========================================
# 4. Core query function (SQL body minimally altered)
# =========================================
def compute_purchase_counts(window_days, start_date, end_date, extra_where):
    query = f"""
 WITH leads_base AS (
      SELECT
        id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
        DATE(lead_submission_date) AS lead_date,
        -- geography_first
        CASE
          WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_first,
        -- geography_last
        CASE
          WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_last
      FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
      WHERE
        lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
        AND useremail IS NOT NULL AND useremail != ''
    ),

    enriched_leads AS (
      SELECT DISTINCT
        *,
        CASE
          WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
          WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
          WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
          ELSE geography_first
        END AS Geography
      FROM leads_base
    ),

    filtered_leads AS (
      SELECT *
      FROM enriched_leads
      WHERE 1=1 {extra_where}
    ),

    cohort_stats AS (
      SELECT COUNT(*) AS total_leads
      FROM filtered_leads
    ),

    purchases_window AS (
      SELECT DISTINCT
        fl.id, fl.useremail, r.received_at, r.subtotal_USD
      FROM filtered_leads fl
      JOIN `kk-data-analytics.TB_Customer_Analytics.Revenue_2025_v1_4` r
        ON r.customer_id = fl.id
       AND DATE(r.received_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)
       AND r.subtotal_USD > 0
    ),

    sl AS (
      SELECT
        COUNT(*)          AS purchases_count,
        SUM(subtotal_USD) AS purchases_revenue
      FROM purchases_window pw
    )

SELECT
  {window_days} AS window_days,
  cs.total_leads,
  sl.purchases_count,
  sl.purchases_revenue,
  SAFE_DIVIDE(sl.purchases_count, cs.total_leads) * 100 AS conversion_rate,
  SAFE_DIVIDE(sl.purchases_revenue, sl.purchases_count) AS avg_revenue_per_purchase
FROM sl
CROSS JOIN cohort_stats cs
    """
    return client.query(query).to_dataframe()



# =========================================
# 3. Helper to build WHERE fragments
# =========================================
def _to_sql_in(values):
    """Return a BigQuery-safe IN (...) string for a Python list."""
    if not values:
        return None
    if isinstance(values[0], (int, float)):
        inner = ",".join(str(v) for v in values)
    else:
        # Use double quotes for string literals as requested
        inner = ",".join('"' + str(v).replace('"', '""') + '"' for v in values)
    return f"({inner})"

def build_extra_where(filters_dict):
    """
    Turn a dict like:
        {'lead_utm_campaign_first': [...], 'is_new_addition': [0,1]}
    into ANDed SQL fragments.
    """
    clauses = []
    for col, vals in filters_dict.items():
        if vals:
            in_clause = _to_sql_in(vals)
            clauses.append(f"{col} IN {in_clause}")
    if clauses:
        return " AND " + " AND ".join(clauses)
    return ""     # no extra filtering


# =========================================
# 4. Runner (no button – executed when this cell runs)
# =========================================
# =========================================
# 5. Runner – combined filters (no per-campaign loop)
# =========================================
def run_batches():
    # parse the date_windows text safely (eval is okay in trusted env; otherwise use json.loads with valid JSON)
    try:
        date_windows = eval(date_windows_text.value, {}, {})
        assert isinstance(date_windows, list)
    except Exception as e:
        raise ValueError(f"Could not parse date_windows_text: {e}")

    filters_dict = get_selected_filters()
    extra_where = build_extra_where(filters_dict)

    all_outputs = []
    for spec in date_windows:
        start_date = spec["start"]
        end_date   = spec["end"]
        windows    = spec["windows"]

        print(f"\n=== Between {start_date} and {end_date} ===")
        print("Filters applied:", filters_dict)

        dfs = [
            compute_purchase_counts(w, start_date, end_date, extra_where)
            for w in windows
        ]
        final_df = pd.concat(dfs, ignore_index=True)

        display(final_df)

        all_outputs.append(((start_date, end_date), final_df.copy()))

    return all_outputs

# Run immediately
results = run_batches()
print("\nDone. `results` holds each (date_range, df) tuple.")


=== Between 2025-07-16 and 2025-07-23 ===
Filters applied: {'lead_utm_campaign_first': ["KK_SEM_IN_B2C-Generic-DevOps_PM_B_PROS_NOA_SEP'24", 'KK_SEM_IN_B2C-Kubernetes-AB_MIX_NB_PROS_DG', "KK_SEM_IN_B2C-Brand_PM_B_PROS_NOA_SEP'24", "KK_SEM_IN_B2C-Generic-Software&Tools_PM_B_PROS_NOA_SEP'24", 'KK_SEM_NA_B2C-Kubernetes-AB_MIX_NB_PROS_DG', 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_GDN_IN_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_GDN_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', 'KK_YT_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', 'KK_SEM_IN_B2C-Non-Brand_MIX_NB_PROS_Air Cover', 'KK_SEM_IN_B2C-Early-Black-Friday_PM_NB_RTG_NOA', 'KK_SEM_IN_B2C-Goodbye-2024_PM_NB_RTG_NOA', 'KK_SEM_IN_B2C-Non-Brand-RLSA_MIX_NB_PROS_Air-Cover', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T2', 'KK_SEM_ROTW_B2C-Goodbye-2024_PM_NB_RTG_NOA_T2', "KK_SEM_UK_B2C-Brand_PM_B_PROS_NOA_SEP'24", 'KK_SEM_UK_B2C-Kubernetes-AB_MIX_NB_PROS_DG', "KK_SEM_US_B2C-Generic-DevOps_PM_NB_PROS_NOA_SEP'24", 'K

,window_days,total_leads,purchases_count,purchases_revenue,conversion_rate,avg_revenue_per_purchase
0,7,239,11,780.391263,4.60251,70.94466



=== Between 2025-07-09 and 2025-07-16 ===
Filters applied: {'lead_utm_campaign_first': ["KK_SEM_IN_B2C-Generic-DevOps_PM_B_PROS_NOA_SEP'24", 'KK_SEM_IN_B2C-Kubernetes-AB_MIX_NB_PROS_DG', "KK_SEM_IN_B2C-Brand_PM_B_PROS_NOA_SEP'24", "KK_SEM_IN_B2C-Generic-Software&Tools_PM_B_PROS_NOA_SEP'24", 'KK_SEM_NA_B2C-Kubernetes-AB_MIX_NB_PROS_DG', 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_GDN_IN_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_GDN_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', 'KK_YT_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', 'KK_SEM_IN_B2C-Non-Brand_MIX_NB_PROS_Air Cover', 'KK_SEM_IN_B2C-Early-Black-Friday_PM_NB_RTG_NOA', 'KK_SEM_IN_B2C-Goodbye-2024_PM_NB_RTG_NOA', 'KK_SEM_IN_B2C-Non-Brand-RLSA_MIX_NB_PROS_Air-Cover', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T2', 'KK_SEM_ROTW_B2C-Goodbye-2024_PM_NB_RTG_NOA_T2', "KK_SEM_UK_B2C-Brand_PM_B_PROS_NOA_SEP'24", 'KK_SEM_UK_B2C-Kubernetes-AB_MIX_NB_PROS_DG', "KK_SEM_US_B2C-Generic-DevOps_PM_NB_PROS_NOA_SEP'24", 'K

,window_days,total_leads,purchases_count,purchases_revenue,conversion_rate,avg_revenue_per_purchase
0,14,217,23,2551.244269,10.599078,110.923664



=== Between 2025-07-02 and 2025-07-09 ===
Filters applied: {'lead_utm_campaign_first': ["KK_SEM_IN_B2C-Generic-DevOps_PM_B_PROS_NOA_SEP'24", 'KK_SEM_IN_B2C-Kubernetes-AB_MIX_NB_PROS_DG', "KK_SEM_IN_B2C-Brand_PM_B_PROS_NOA_SEP'24", "KK_SEM_IN_B2C-Generic-Software&Tools_PM_B_PROS_NOA_SEP'24", 'KK_SEM_NA_B2C-Kubernetes-AB_MIX_NB_PROS_DG', 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_GDN_IN_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_GDN_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', 'KK_YT_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', 'KK_SEM_IN_B2C-Non-Brand_MIX_NB_PROS_Air Cover', 'KK_SEM_IN_B2C-Early-Black-Friday_PM_NB_RTG_NOA', 'KK_SEM_IN_B2C-Goodbye-2024_PM_NB_RTG_NOA', 'KK_SEM_IN_B2C-Non-Brand-RLSA_MIX_NB_PROS_Air-Cover', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T2', 'KK_SEM_ROTW_B2C-Goodbye-2024_PM_NB_RTG_NOA_T2', "KK_SEM_UK_B2C-Brand_PM_B_PROS_NOA_SEP'24", 'KK_SEM_UK_B2C-Kubernetes-AB_MIX_NB_PROS_DG', "KK_SEM_US_B2C-Generic-DevOps_PM_NB_PROS_NOA_SEP'24", 'K

,window_days,total_leads,purchases_count,purchases_revenue,conversion_rate,avg_revenue_per_purchase
0,21,280,20,1905.875314,7.142857,95.293766



=== Between 2025-06-25 and 2025-07-02 ===
Filters applied: {'lead_utm_campaign_first': ["KK_SEM_IN_B2C-Generic-DevOps_PM_B_PROS_NOA_SEP'24", 'KK_SEM_IN_B2C-Kubernetes-AB_MIX_NB_PROS_DG', "KK_SEM_IN_B2C-Brand_PM_B_PROS_NOA_SEP'24", "KK_SEM_IN_B2C-Generic-Software&Tools_PM_B_PROS_NOA_SEP'24", 'KK_SEM_NA_B2C-Kubernetes-AB_MIX_NB_PROS_DG', 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_GDN_IN_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_GDN_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', 'KK_YT_IN_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA', 'KK_SEM_IN_B2C-Non-Brand_MIX_NB_PROS_Air Cover', 'KK_SEM_IN_B2C-Early-Black-Friday_PM_NB_RTG_NOA', 'KK_SEM_IN_B2C-Goodbye-2024_PM_NB_RTG_NOA', 'KK_SEM_IN_B2C-Non-Brand-RLSA_MIX_NB_PROS_Air-Cover', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T2', 'KK_SEM_ROTW_B2C-Goodbye-2024_PM_NB_RTG_NOA_T2', "KK_SEM_UK_B2C-Brand_PM_B_PROS_NOA_SEP'24", 'KK_SEM_UK_B2C-Kubernetes-AB_MIX_NB_PROS_DG', "KK_SEM_US_B2C-Generic-DevOps_PM_NB_PROS_NOA_SEP'24", 'K

,window_days,total_leads,purchases_count,purchases_revenue,conversion_rate,avg_revenue_per_purchase
0,28,445,35,3203.040362,7.865169,91.515439



Done. `results` holds each (date_range, df) tuple.


In [ ]:
# =========================================
# 0. Imports & BigQuery client
# =========================================
import pandas as pd
from google.cloud import bigquery
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

client = bigquery.Client(project="kk-data-analytics")  # change project if needed


# =========================================
# 1. Campaign selector UI  (run this cell first)
# =========================================
campaign_query = """
  SELECT DISTINCT lead_utm_campaign_first
  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
  WHERE lead_utm_campaign_first LIKE 'KK%'
  AND lead_utm_campaign_first NOT LIKE '%FBA%' AND lead_utm_campaign_first LIKE '%PMAX%' AND lead_utm_campaign_first NOT LIKE '%Sale%'
  -- AND lead_submission_date BETWEEN TIMESTAMP('2025-01-01') AND TIMESTAMP('2025-07-23')
"""
df_new_filters = client.query(campaign_query).to_dataframe()

selectors = {}
campaign_options = df_new_filters['lead_utm_campaign_first'].dropna().tolist()

campaign_widget = widgets.SelectMultiple(
    options=campaign_options,
    value=campaign_options,               # preselect all
    description='Campaigns',
    rows=min(12, len(campaign_options)),
    layout=Layout(width='60%')
)
display(campaign_widget)
selectors['lead_utm_campaign_first'] = campaign_widget

new_addition_widget = widgets.SelectMultiple(
    options=[0, 1],
    value=[],                             # none selected => ignore this field
    description='New Lead?',
    rows=2,
    layout=Layout(width='20%')
)
display(new_addition_widget)
selectors['is_new_addition'] = new_addition_widget

def get_selected_filters():
    return {
        'lead_utm_campaign_first': list(selectors['lead_utm_campaign_first'].value),
        'is_new_addition': list(selectors['is_new_addition'].value)
    }


SelectMultiple(description='Campaigns', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1…

SelectMultiple(description='New\xa0Lead?', layout=Layout(width='20%'), options=(0, 1), rows=2, value=())

In [ ]:
selected_filters = get_selected_filters()
print(selected_filters)

{'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_PMAX_IN_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_UK_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_UK_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_UK_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_Generic-DevOps_CONV_B_PROS_MULT_NA', 

In [ ]:
# =========================================
# 3. Core query function (SQL body unchanged)
# =========================================
# =========================================
# 4. Core query function (SQL body minimally altered)
# =========================================
def compute_purchase_counts(window_days, start_date, end_date, extra_where):
    query = f"""
 WITH leads_base AS (
      SELECT
        id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
        DATE(lead_submission_date) AS lead_date,
        -- geography_first
        CASE
          WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_first,
        -- geography_last
        CASE
          WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_last
      FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
      WHERE
        lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
        AND useremail IS NOT NULL AND useremail != ''
    ),

    enriched_leads AS (
      SELECT DISTINCT
        *,
        CASE
          WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
          WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
          WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
          ELSE geography_first
        END AS Geography
      FROM leads_base
    ),

    filtered_leads AS (
      SELECT *
      FROM enriched_leads
      WHERE 1=1 {extra_where}
    ),

    cohort_stats AS (
      SELECT COUNT(*) AS total_leads
      FROM filtered_leads
    ),

    purchases_window AS (
      SELECT DISTINCT
        fl.id, fl.useremail, r.received_at, r.subtotal_USD
      FROM filtered_leads fl
      JOIN `kk-data-analytics.TB_Customer_Analytics.Revenue_2025_v1_4` r
        ON r.customer_id = fl.id
       AND DATE(r.received_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)
       AND r.subtotal_USD > 0
    ),

    sl AS (
      SELECT
        COUNT(*)          AS purchases_count,
        SUM(subtotal_USD) AS purchases_revenue
      FROM purchases_window pw
    )

SELECT
  {window_days} AS window_days,
  cs.total_leads,
  sl.purchases_count,
  sl.purchases_revenue,
  SAFE_DIVIDE(sl.purchases_count, cs.total_leads) * 100 AS conversion_rate,
  SAFE_DIVIDE(sl.purchases_revenue, sl.purchases_count) AS avg_revenue_per_purchase
FROM sl
CROSS JOIN cohort_stats cs
    """
    return client.query(query).to_dataframe()



# =========================================
# 3. Helper to build WHERE fragments
# =========================================
def _to_sql_in(values):
    """Return a BigQuery-safe IN (...) string for a Python list."""
    if not values:
        return None
    if isinstance(values[0], (int, float)):
        inner = ",".join(str(v) for v in values)
    else:
        # Use double quotes for string literals as requested
        inner = ",".join('"' + str(v).replace('"', '""') + '"' for v in values)
    return f"({inner})"

def build_extra_where(filters_dict):
    """
    Turn a dict like:
        {'lead_utm_campaign_first': [...], 'is_new_addition': [0,1]}
    into ANDed SQL fragments.
    """
    clauses = []
    for col, vals in filters_dict.items():
        if vals:
            in_clause = _to_sql_in(vals)
            clauses.append(f"{col} IN {in_clause}")
    if clauses:
        return " AND " + " AND ".join(clauses)
    return ""     # no extra filtering


# =========================================
# 4. Runner (no button – executed when this cell runs)
# =========================================
# =========================================
# 5. Runner – combined filters (no per-campaign loop)
# =========================================
def run_batches():
    # parse the date_windows text safely (eval is okay in trusted env; otherwise use json.loads with valid JSON)
    try:
        date_windows = eval(date_windows_text.value, {}, {})
        assert isinstance(date_windows, list)
    except Exception as e:
        raise ValueError(f"Could not parse date_windows_text: {e}")

    filters_dict = get_selected_filters()
    extra_where = build_extra_where(filters_dict)

    all_outputs = []
    for spec in date_windows:
        start_date = spec["start"]
        end_date   = spec["end"]
        windows    = spec["windows"]

        print(f"\n=== Between {start_date} and {end_date} ===")
        print("Filters applied:", filters_dict)

        dfs = [
            compute_purchase_counts(w, start_date, end_date, extra_where)
            for w in windows
        ]
        final_df = pd.concat(dfs, ignore_index=True)

        display(final_df)

        all_outputs.append(((start_date, end_date), final_df.copy()))

    return all_outputs

# Run immediately
results = run_batches()
print("\nDone. `results` holds each (date_range, df) tuple.")


=== Between 2025-07-16 and 2025-07-23 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_PMAX_IN_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_UK_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_UK_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_UK_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_NAN_PROS

,window_days,total_leads,purchases_count,purchases_revenue,conversion_rate,avg_revenue_per_purchase
0,7,110,2,175.796,1.818182,87.898



=== Between 2025-07-09 and 2025-07-16 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_PMAX_IN_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_UK_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_UK_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_UK_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_NAN_PROS

,window_days,total_leads,purchases_count,purchases_revenue,conversion_rate,avg_revenue_per_purchase
0,14,81,5,631.991281,6.17284,126.398256



=== Between 2025-07-02 and 2025-07-09 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_PMAX_IN_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_UK_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_UK_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_UK_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_NAN_PROS

,window_days,total_leads,purchases_count,purchases_revenue,conversion_rate,avg_revenue_per_purchase
0,21,335,3,132.477764,0.895522,44.159255



=== Between 2025-06-25 and 2025-07-02 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_IN_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_PMAX_IN_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_IN_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_UK_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_UK_B2C-Kubernetes_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_UK_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_Generic-DevOps_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-DevOps_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_B_PROS_MULT_NA', 'KK_PMAX_US_B2C-Kubernetes_CONV_NAN_PROS

,window_days,total_leads,purchases_count,purchases_revenue,conversion_rate,avg_revenue_per_purchase
0,28,536,3,310.912767,0.559701,103.637589



Done. `results` holds each (date_range, df) tuple.


In [ ]:
# =========================================
# 0. Imports & BigQuery client
# =========================================
import pandas as pd
from google.cloud import bigquery
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display

client = bigquery.Client(project="kk-data-analytics")  # change project if needed


# =========================================
# 1. Campaign selector UI  (run this cell first)
# =========================================
campaign_query = """
  SELECT DISTINCT lead_utm_campaign_first
  FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
  WHERE lead_utm_campaign_first LIKE 'KK%'
  AND lead_utm_campaign_first NOT LIKE '%FBA%' AND lead_utm_campaign_first LIKE '%ROTW%'
  -- AND lead_submission_date BETWEEN TIMESTAMP('2025-01-01') AND TIMESTAMP('2025-07-23')
"""
df_new_filters = client.query(campaign_query).to_dataframe()

selectors = {}
campaign_options = df_new_filters['lead_utm_campaign_first'].dropna().tolist()

campaign_widget = widgets.SelectMultiple(
    options=campaign_options,
    value=campaign_options,               # preselect all
    description='Campaigns',
    rows=min(12, len(campaign_options)),
    layout=Layout(width='60%')
)
display(campaign_widget)
selectors['lead_utm_campaign_first'] = campaign_widget

new_addition_widget = widgets.SelectMultiple(
    options=[0, 1],
    value=[],                             # none selected => ignore this field
    description='New Lead?',
    rows=2,
    layout=Layout(width='20%')
)
display(new_addition_widget)
selectors['is_new_addition'] = new_addition_widget

def get_selected_filters():
    return {
        'lead_utm_campaign_first': list(selectors['lead_utm_campaign_first'].value),
        'is_new_addition': list(selectors['is_new_addition'].value)
    }


SelectMultiple(description='Campaigns', index=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1…

SelectMultiple(description='New\xa0Lead?', layout=Layout(width='20%'), options=(0, 1), rows=2, value=())

In [ ]:
selected_filters = get_selected_filters()
print(selected_filters)

{'lead_utm_campaign_first': ['KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_PMAX_ROTW_B2C-Sale-Campaign_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_ROTW_B2C-MegaMay-Sale_CONV_B_PROS_MULT_NA', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T2', 'KK_SEM_ROTW_B2C-Goodbye-2024_PM_NB_RTG_NOA_T2', 'KK_YT_ROTW_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_YT_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_SEM_ROTW_B2B-DevOps_PM_NB_PROS_NOA_NA', 'KK_PMAX_ROTW_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_SEM_ROTW_B2C-Free-Week-2025_PM_NB_RTG_NOA', 'KK_PMAX_ROTW_B2C-Free-Week-2025_CONV_NAN_PROS_MULT_NA', 'KK_GDN_ROTW_B2C-Free-Week-2025_CONV_NAN_RTG_MULT_NA', 'KK_GDN_ROTW_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T1', 'KK_SEM_ROTW_B2C-Sale-Campaign_PM_NB_RTG_NOA', 'KK_YT_ROTW_B2C-Free-Week-2025_CONV_NAN_RTG_MULT_NA'

In [ ]:
# =========================================
# 3. Core query function (SQL body unchanged)
# =========================================
# =========================================
# 4. Core query function (SQL body minimally altered)
# =========================================
def compute_purchase_counts(window_days, start_date, end_date, extra_where):
    query = f"""
 WITH leads_base AS (
      SELECT
        id,
        lead_utm_campaign_first,
        lead_utm_campaign_latest,
        is_new_addition,
        utm_source_first,
        utm_source_latest,
        useremail,
        DATE(lead_submission_date) AS lead_date,
        -- geography_first
        CASE
          WHEN lead_utm_campaign_first IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_first, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_first,
        -- geography_last
        CASE
          WHEN lead_utm_campaign_latest IS NULL THEN 'TBU'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_IN_') THEN 'India'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_US_') THEN 'US'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_UK_') THEN 'UK'
          WHEN REGEXP_CONTAINS(lead_utm_campaign_latest, r'_ROTW_|ROTW|ROW') THEN 'ROTW'
          ELSE 'TBU'
        END AS geography_last
      FROM `kk-data-analytics.TB_Customer_Analytics.Lead_List_2025_v4_5`
      WHERE
        lead_submission_date BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}')
        AND useremail IS NOT NULL AND useremail != ''
    ),

    enriched_leads AS (
      SELECT DISTINCT
        *,
        CASE
          WHEN geography_first = 'TBU' AND geography_last != 'TBU' THEN geography_last
          WHEN geography_last = 'TBU' AND geography_first != 'TBU' THEN geography_first
          WHEN geography_first = 'TBU' AND geography_last = 'TBU' THEN 'TBU'
          ELSE geography_first
        END AS Geography
      FROM leads_base
    ),

    filtered_leads AS (
      SELECT *
      FROM enriched_leads
      WHERE 1=1 {extra_where}
    ),

    cohort_stats AS (
      SELECT COUNT(*) AS total_leads
      FROM filtered_leads
    ),

    purchases_window AS (
      SELECT DISTINCT
        fl.id, fl.useremail, r.received_at, r.subtotal_USD
      FROM filtered_leads fl
      JOIN `kk-data-analytics.TB_Customer_Analytics.Revenue_2025_v1_4` r
        ON r.customer_id = fl.id
       AND DATE(r.received_at)
           BETWEEN fl.lead_date
               AND DATE_ADD(fl.lead_date, INTERVAL {window_days} DAY)
       AND r.subtotal_USD > 0
    ),

    sl AS (
      SELECT
        COUNT(*)          AS purchases_count,
        SUM(subtotal_USD) AS purchases_revenue
      FROM purchases_window pw
    )

SELECT
  {window_days} AS window_days,
  cs.total_leads,
  sl.purchases_count,
  sl.purchases_revenue,
  SAFE_DIVIDE(sl.purchases_count, cs.total_leads) * 100 AS conversion_rate,
  SAFE_DIVIDE(sl.purchases_revenue, sl.purchases_count) AS avg_revenue_per_purchase
FROM sl
CROSS JOIN cohort_stats cs
    """
    return client.query(query).to_dataframe()



# =========================================
# 3. Helper to build WHERE fragments
# =========================================
def _to_sql_in(values):
    """Return a BigQuery-safe IN (...) string for a Python list."""
    if not values:
        return None
    if isinstance(values[0], (int, float)):
        inner = ",".join(str(v) for v in values)
    else:
        # Use double quotes for string literals as requested
        inner = ",".join('"' + str(v).replace('"', '""') + '"' for v in values)
    return f"({inner})"

def build_extra_where(filters_dict):
    """
    Turn a dict like:
        {'lead_utm_campaign_first': [...], 'is_new_addition': [0,1]}
    into ANDed SQL fragments.
    """
    clauses = []
    for col, vals in filters_dict.items():
        if vals:
            in_clause = _to_sql_in(vals)
            clauses.append(f"{col} IN {in_clause}")
    if clauses:
        return " AND " + " AND ".join(clauses)
    return ""     # no extra filtering


# =========================================
# 4. Runner (no button – executed when this cell runs)
# =========================================
# =========================================
# 5. Runner – combined filters (no per-campaign loop)
# =========================================
def run_batches():
    # parse the date_windows text safely (eval is okay in trusted env; otherwise use json.loads with valid JSON)
    try:
        date_windows = eval(date_windows_text.value, {}, {})
        assert isinstance(date_windows, list)
    except Exception as e:
        raise ValueError(f"Could not parse date_windows_text: {e}")

    filters_dict = get_selected_filters()
    extra_where = build_extra_where(filters_dict)

    all_outputs = []
    for spec in date_windows:
        start_date = spec["start"]
        end_date   = spec["end"]
        windows    = spec["windows"]

        print(f"\n=== Between {start_date} and {end_date} ===")
        print("Filters applied:", filters_dict)

        dfs = [
            compute_purchase_counts(w, start_date, end_date, extra_where)
            for w in windows
        ]
        final_df = pd.concat(dfs, ignore_index=True)

        display(final_df)

        all_outputs.append(((start_date, end_date), final_df.copy()))

    return all_outputs

# Run immediately
results = run_batches()
print("\nDone. `results` holds each (date_range, df) tuple.")


=== Between 2025-07-16 and 2025-07-23 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_PMAX_ROTW_B2C-Sale-Campaign_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_ROTW_B2C-MegaMay-Sale_CONV_B_PROS_MULT_NA', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T2', 'KK_SEM_ROTW_B2C-Goodbye-2024_PM_NB_RTG_NOA_T2', 'KK_YT_ROTW_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_YT_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_SEM_ROTW_B2B-DevOps_PM_NB_PROS_NOA_NA', 'KK_PMAX_ROTW_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_SEM_ROTW_B2C-Free-Week-2025_PM_NB_RTG_NOA', 'KK_PMAX_ROTW_B2C-Free-Week-2025_CONV_NAN_PROS_MULT_NA', 'KK_GDN_ROTW_B2C-Free-Week-2025_CONV_NAN_RTG_MULT_NA', 'KK_GDN_ROTW_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T1', 'KK_SEM_ROTW_B2C-Sale-Campaign_PM_NB_RT

,window_days,total_leads,purchases_count,purchases_revenue,conversion_rate,avg_revenue_per_purchase
0,7,7,0,NaN,0.0,NaN



=== Between 2025-07-09 and 2025-07-16 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_PMAX_ROTW_B2C-Sale-Campaign_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_ROTW_B2C-MegaMay-Sale_CONV_B_PROS_MULT_NA', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T2', 'KK_SEM_ROTW_B2C-Goodbye-2024_PM_NB_RTG_NOA_T2', 'KK_YT_ROTW_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_YT_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_SEM_ROTW_B2B-DevOps_PM_NB_PROS_NOA_NA', 'KK_PMAX_ROTW_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_SEM_ROTW_B2C-Free-Week-2025_PM_NB_RTG_NOA', 'KK_PMAX_ROTW_B2C-Free-Week-2025_CONV_NAN_PROS_MULT_NA', 'KK_GDN_ROTW_B2C-Free-Week-2025_CONV_NAN_RTG_MULT_NA', 'KK_GDN_ROTW_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T1', 'KK_SEM_ROTW_B2C-Sale-Campaign_PM_NB_RT

,window_days,total_leads,purchases_count,purchases_revenue,conversion_rate,avg_revenue_per_purchase
0,14,7,2,285.0,28.571429,142.5



=== Between 2025-07-02 and 2025-07-09 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_PMAX_ROTW_B2C-Sale-Campaign_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_ROTW_B2C-MegaMay-Sale_CONV_B_PROS_MULT_NA', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T2', 'KK_SEM_ROTW_B2C-Goodbye-2024_PM_NB_RTG_NOA_T2', 'KK_YT_ROTW_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_YT_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_SEM_ROTW_B2B-DevOps_PM_NB_PROS_NOA_NA', 'KK_PMAX_ROTW_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_SEM_ROTW_B2C-Free-Week-2025_PM_NB_RTG_NOA', 'KK_PMAX_ROTW_B2C-Free-Week-2025_CONV_NAN_PROS_MULT_NA', 'KK_GDN_ROTW_B2C-Free-Week-2025_CONV_NAN_RTG_MULT_NA', 'KK_GDN_ROTW_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T1', 'KK_SEM_ROTW_B2C-Sale-Campaign_PM_NB_RT

,window_days,total_leads,purchases_count,purchases_revenue,conversion_rate,avg_revenue_per_purchase
0,21,14,1,360.0,7.142857,360.0



=== Between 2025-06-25 and 2025-07-02 ===
Filters applied: {'lead_utm_campaign_first': ['KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T2', 'KK_GDN_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_PMAX_ROTW_B2C-Sale-Campaign_CONV_NAN_PROS_MULT_NA', 'KK_PMAX_ROTW_B2C-Goodbye-2024_CONV_NAN_PROS_MULT_NA_T1', 'KK_PMAX_ROTW_B2C-MegaMay-Sale_CONV_B_PROS_MULT_NA', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T2', 'KK_SEM_ROTW_B2C-Goodbye-2024_PM_NB_RTG_NOA_T2', 'KK_YT_ROTW_B2C-Early-Black-Friday_CONV_NAN_RTG_MULT_NA', 'KK_YT_ROTW_B2C-Goodbye-2024_CONV_NAN_RTG_MULT_NA_T2', 'KK_SEM_ROTW_B2B-DevOps_PM_NB_PROS_NOA_NA', 'KK_PMAX_ROTW_B2C-Early-Black-Friday_CONV_NAN_PROS_MULT_NA', 'KK_SEM_ROTW_B2C-Free-Week-2025_PM_NB_RTG_NOA', 'KK_PMAX_ROTW_B2C-Free-Week-2025_CONV_NAN_PROS_MULT_NA', 'KK_GDN_ROTW_B2C-Free-Week-2025_CONV_NAN_RTG_MULT_NA', 'KK_GDN_ROTW_B2C-Sale-Campaign_CONV_NAN_RTG_MULT_NA', 'KK_SEM_ROTW_B2C-Early-Black-Friday_PM_NB_RTG_NOA_T1', 'KK_SEM_ROTW_B2C-Sale-Campaign_PM_NB_RT

,window_days,total_leads,purchases_count,purchases_revenue,conversion_rate,avg_revenue_per_purchase
0,28,18,1,96.79725,5.555556,96.79725



Done. `results` holds each (date_range, df) tuple.
